### Import Statement

In [ ]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
import kagglehub


path = kagglehub.dataset_download("arpitjain007/dog-vs-cat-fastai")
print("Path to dataset files:", path)


Path to dataset files: /kaggle/input/dog-vs-cat-fastai


In [ ]:
directory = r"/kaggle/input/dog-vs-cat-fastai/dogscats/train"

In [ ]:
IMAGE_SIZE = (160, 160)
BATCH_SIZE = 32

train_data = image_dataset_from_directory(
    directory,
    validation_split=0.1,
    subset="training",
    seed=42,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int"
)

val_data = image_dataset_from_directory(
    directory,
    validation_split=0.1,
    subset="validation",
    seed=42,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int"
)


Found 23000 files belonging to 2 classes.
Using 20700 files for training.
Found 23000 files belonging to 2 classes.
Using 2300 files for validation.


In [ ]:
from tensorflow.keras import layers

normalization_layer = layers.Rescaling(1./255)

train_data = train_data.map(lambda x, y: (normalization_layer(x), y))
val_data = val_data.map(lambda x, y: (normalization_layer(x), y))



In [ ]:
from tensorflow.keras import models

data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])
train_data = train_data.map(lambda x, y: (data_augmentation(x, training=True), y))



In [ ]:
from tensorflow.keras import applications

base_model = applications.MobileNetV2(
    input_shape=IMAGE_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False



9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(), #mean of entire feature map (similar to flatten)
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(2, activation='softmax')  # 2 classes
])


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
callback = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
model.fit(train_data, validation_data=val_data, epochs=10, callbacks=[callback])


Epoch 1/10
647/647 ━━━━━━━━━━━━━━━━━━━━ 157s 226ms/step - accuracy: 0.9211 - loss: 0.1930 - val_accuracy: 0.9791 - val_loss: 0.0582
Epoch 2/10
647/647 ━━━━━━━━━━━━━━━━━━━━ 182s 205ms/step - accuracy: 0.9515 - loss: 0.1150 - val_accuracy: 0.9809 - val_loss: 0.0549
Epoch 3/10
647/647 ━━━━━━━━━━━━━━━━━━━━ 134s 207ms/step - accuracy: 0.9584 - loss: 0.1056 - val_accuracy: 0.9804 - val_loss: 0.0508
Epoch 4/10
647/647 ━━━━━━━━━━━━━━━━━━━━ 140s 204ms/step - accuracy: 0.9582 - loss: 0.1001 - val_accuracy: 0.9787 - val_loss: 0.0531
Epoch 5/10
647/647 ━━━━━━━━━━━━━━━━━━━━ 132s 203ms/step - accuracy: 0.9614 - loss: 0.0989 - val_accuracy: 0.9770 - val_loss: 0.0593


In [ ]:
model.save("Cat_vs_Dog.h5")